In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from lbg_desc_forecast import get_lbg_mappers, fig_dir

In [ ]:
contamination = 0.1

In [ ]:
u1, g1, r1 = get_lbg_mappers(1)
u10, g10, r10 = get_lbg_mappers(10)


def calc_spectra(mapper):
    ell, Cgg, Ckg, Ckk, Cff = mapper.calc_spectra()
    _, Cmag = mapper.auto_mag_bias()
    _, Cff_raw = mapper.auto_Cff(raw=True)
    Ngg = mapper.shot_noise
    f_sky = mapper.f_sky
    return dict(
        ell=ell,
        Cgg=Cgg,
        Cmag=Cmag,
        Cff=Cff,
        Cff_raw=Cff_raw,
        Ngg=Ngg,
        f_sky=f_sky,
    )


spectra = {
    1: dict(
        u=calc_spectra(u1),
        g=calc_spectra(g1),
        r=calc_spectra(r1),
    ),
    10: dict(
        u=calc_spectra(u10),
        g=calc_spectra(g10),
        r=calc_spectra(r10),
    ),
}

# We will delete some points because I think they
# have integration errors and are ugly on the plot
for row in [[1, "r", 1132], [10, "g", 2637]]:
    year, band, idx = row
    for key, val in spectra[year][band].items():
        if isinstance(val, np.ndarray):
            spectra[year][band][key] = np.delete(val, idx)

$$
\mathrm{Cov}^{gg,gg}
= \frac{2}{(2 \ell + 1)f_\text{sky}} (C_\ell^{gg} + N_\ell^{gg})^2

\quad \to \quad

\sqrt{\mathrm{Cov}^{gg,gg}}
= \left( \frac{2}{(2 \ell + 1)f_\text{sky}} \right)^{1/2} (C_\ell^{gg} + N_\ell^{gg})
$$


In [ ]:
fig, axes = plt.subplots(
    2,
    3,
    figsize=(4.5, 3.5),
    constrained_layout=True,
    dpi=150,
    sharex=True,
    sharey=True,
)

for row, year in zip(axes, [1, 10]):
    for ax, band, color in zip(row, "ugr", ["C4", "C0", "C2"]):
        # Unpack spectra
        ell = spectra[year][band]["ell"]
        Cgg = spectra[year][band]["Cgg"]
        Cmag = spectra[year][band]["Cmag"]
        Cff = spectra[year][band]["Cff"] * contamination
        Cff_raw = spectra[year][band]["Cff_raw"] * contamination
        Ngg = spectra[year][band]["Ngg"]
        f_sky = spectra[year][band]["f_sky"]

        # Calculate noise weight
        w = np.sqrt(2 / (2 * ell + 1) / f_sky)

        # Plot spectrum and noise terms
        ax.plot(ell, Cgg, c=color)
        ax.plot(ell, w * Cgg, c="silver")
        ax.plot(ell, w * Cmag, c="silver", ls="-.")
        ax.plot(ell, w * Cff_raw, c=color, alpha=0.5)
        ax.plot(ell, w * Cff, c=color, ls="--")
        ax.plot(ell, w * np.full_like(ell, Ngg), c=color, ls=":")

        # Gray out excluded ell regions
        ax.fill_betweenx(
            [0, 1e-3], [0, 0], [50, 50], color="silver", alpha=0.3, zorder=0, lw=0
        )
        ax.fill_betweenx(
            [0, 1e-3], [2e3, 2e3], [1e5, 1e5], color="silver", alpha=0.3, zorder=0, lw=0
        )

        ax.set(xlim=(10, ell.max()), ylim=(1e-11, 1e-4))
        ax.text(
            0.5,
            0.89,
            f"year {year}",
            transform=ax.transAxes,
            ha="center",
            va="bottom",
        )

    row[0].set(yscale="log", xscale="log", ylabel="$C_\ell$")

axes[0, 0].set_title("$u$ dropouts ($z \sim 3$)")
axes[0, 1].set_title("$g$ dropouts ($z \sim 4$)")
axes[0, 2].set_title("$r$ dropouts ($z \sim 5$)")
for ax in axes[1]:
    ax.set_xlabel(xlabel="$\ell$")

fig.savefig(fig_dir / "noise_spectra.pdf", bbox_inches="tight")